# Testes de classificação nos dados sociodemograficos;

## Modificação da base de dados de acordo com a demanda do projeto;

### Importando as bibliotecas necessárias;

In [1]:
#Importando as bibliotecas necessárias para analisar os dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import openpyxl.utils.cell as cell


### Importando os dados e manipulando-os;

In [2]:
#Importando os dados
data_frame = pd.read_excel('BANCO DE DADOS - INVERTIDO - 24 de Agosto 2019.xlsx')


#Renoemando as colunas para seguir o padrão do excel
def number_to_excel_column(num):
    """
    Converte um número de coluna em uma string com a representação alfabética.
    Exemplo: 1 => 'A', 2 => 'B', ..., 26 => 'Z', 27 => 'AA', 28 => 'AB', ...
    """
    return cell.get_column_letter(num)

# Gera uma lista com os nomes das colunas no estilo do Excel
col_names = [number_to_excel_column(num) for num in range(1, len(data_frame.columns)+1)]

# Cria um dicionário com os nomes atuais e os novos nomes das colunas
new_col_names = {old_name: new_name for old_name, new_name in zip(data_frame.columns, col_names)}

# Renomeia as colunas
data_frame = data_frame.rename(columns=new_col_names)

data_frame_r = data_frame.replace('', np.nan)


In [3]:
#Selecionando a parte de dados sociodemograficos;
df_socioeconomico = data_frame_r.loc[:, 'B':'BP']

# Parte do dataframe que contem o questionário WHOQOL
df_whoqol = data_frame_r.loc[:, 'BR':'CU']

# Parte do dataframe que contem o questionário de MHI
df_mhi = data_frame_r.loc[:, 'CW':'EH']


In [4]:
print(df_socioeconomico.head())
print(df_mhi.head())

     B     C    D    E     F     G    H    I    J    K  ...   BG   BH   BI  \
0  1.0  25.0  1.0  1.0   1.0   1.0  0.0  1.0  1.0  1.0  ...  0.0  0.0  0.0   
1  1.0  19.0  1.0  1.0   1.0  83.0  1.0  1.0  1.0  1.0  ...  0.0  0.0  0.0   
2  1.0  28.0  5.0  1.0  23.0  23.0  1.0  1.0  1.0  1.0  ...  0.0  0.0  0.0   
3  1.0  22.0  1.0  1.0  37.0   1.0  2.0  1.0  1.0  1.0  ...  0.0  0.0  0.0   
4  1.0  21.0  1.0  1.0   6.0   1.0  2.0  3.0  5.0  1.0  ...  0.0  0.0  0.0   

    BJ   BK   BL   BM   BN   BO   BP  
0  0.0  0.0  0.0  0.0  0.0  0.0  8.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  7.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  7.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  6.0  
4  0.0  0.0  0.0  0.0  0.0  1.0  7.0  

[5 rows x 67 columns]
    CW   CX   CY   CZ   DA   DB   DC   DD   DE   DF  ...   DY   DZ   EA   EB  \
0  4.0  4.0  4.0  6.0  4.0  4.0  4.0  5.0  5.0  3.0  ...  5.0  4.0  4.0  4.0   
1  5.0  5.0  4.0  4.0  4.0  4.0  2.0  3.0  1.0  4.0  ...  2.0  2.0  4.0  5.0   
2  3.0  4.0  1.0  3.0  3.0  2.0  3

### Calculando os niveis de saude mental do MHI;
Para calcular o "score" é a soma de cada um dos items do questionario MHI.
Os niveis de saude mental são definidos da seguinte forma:
- Nivel baixo: (Media - 1*Desvio Padrão)
- Nivel medio: (Media)
- Nivel alto: (Media + 1*Desvio Padrão)

In [5]:
#Calculando os niveis de saude mental do MHI
df_mhi['MHI_Score'] = df_mhi.sum(axis=1)

#Calculando a media e o desvio padrão
media = df_mhi['MHI_Score'].mean()
desvio_padrao = df_mhi['MHI_Score'].std()

#Calculando os niveis de saude mental
df_mhi['Nivel_MHI'] = df_mhi['MHI_Score'].apply(lambda x: 'Baixo' if x < (media - desvio_padrao) else ('Medio' if x < media + desvio_padrao else 'Alto'))


### Aplicando as modificações na base de dados;

In [6]:
# Definindo as colunas que serão removidas:
rem_col_names = [ # Nome
                 'G', # Local de nascimento [Temporariamente] <-
                 'K', # Curso de graduação [Temporariamente] <-
                 'P', # Indique os princiapis fatores de estresse
                 'Q', 'R', 'S','T', # Fatores de estresse
                 'V', # Qual o motivo do afastamento
                 'W', 'X','Y','Z', # Motivos do afastamento
                 'AF', # Caso receba auxilio...
                 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', # Auxilio
                 'BA', # Faz uso continuo de algum medicamento
                 'BB', 'BC', # Medicamentos
                 'BE',# Qual substancia psicoativa
                 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BN' # Substancias psicoativas
                 ]


# Removendo as colunas
df_socioeconomico = df_socioeconomico.drop(columns=rem_col_names, axis=1)

In [7]:
# Modificando os dados das colunas:

# Coluna 'D' - Raça; Varia de 1 à 5 e será aplicado one-hot-encoding, ou seja, será criada uma coluna para cada valor.
oh_raca = pd.get_dummies(df_socioeconomico['D'], prefix='D')
df_socioeconomico = df_socioeconomico.drop('D', axis=1)
df_socioeconomico = df_socioeconomico.join(oh_raca)

# Coluna 'E' - Situacao Civil; Aplicando one-hot-encoding
oh_situacao_civil = pd.get_dummies(df_socioeconomico['E'], prefix='E')
df_socioeconomico = df_socioeconomico.drop('E', axis=1)
df_socioeconomico = df_socioeconomico.join(oh_situacao_civil)

# Coluna 'F' - Local de Origem; Se o local for diferente de '1' (Sobral) será substituido por '0' (Outro)
df_socioeconomico['F'] = df_socioeconomico['F'].apply(lambda x: 0 if x != 1 else x)

# Coluna 'I' - Com quem reside; Aplicando one-hot-encoding
oh_reside = pd.get_dummies(df_socioeconomico['I'], prefix='I')
df_socioeconomico = df_socioeconomico.drop('I', axis=1)
df_socioeconomico = df_socioeconomico.join(oh_reside)

# Coluna 'J' - Religiao; Aplicando one-hot-encoding
oh_religiao = pd.get_dummies(df_socioeconomico['J'], prefix='J')
df_socioeconomico = df_socioeconomico.drop('J', axis=1)
df_socioeconomico = df_socioeconomico.join(oh_religiao)

# Coluna 'AA' - Turno de estudo; Aplicando one-hot-encoding
oh_turno = pd.get_dummies(df_socioeconomico['AA'], prefix='AA')
df_socioeconomico = df_socioeconomico.drop('AA', axis=1)
df_socioeconomico = df_socioeconomico.join(oh_turno)

# Coluna 'AB' - Transporte; Aplicando one-hot-encoding
oh_transporte = pd.get_dummies(df_socioeconomico['AB'], prefix='AB')
df_socioeconomico = df_socioeconomico.drop('AB', axis=1)
df_socioeconomico = df_socioeconomico.join(oh_transporte)


In [8]:
# Unindo o dataframe socioeconomico com a coluna de niveis de saude mental do MHI
df_socioeconomico = df_socioeconomico.join(df_mhi['Nivel_MHI'])
# Removendo as linhas que contem valores nulos
df_socioeconomico = df_socioeconomico.dropna()


### Salvando o dataframe em um arquivo csv;

In [9]:
# Salvando o dataframe em um arquivo csv
df_socioeconomico.to_csv('dataframe_socioeconomico_Niveis.csv', index=False)

## Aplicando alguns modelos de classificação para prever os niveis de saude mental;
(1) SVM
(2) Random Forest
(3) MLP
(4) XGBoost

In [10]:
# Importando as bibliotecas necessárias
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Importando os modelos de classificação
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier


In [11]:
# Separando os dados de treino e teste
X = df_socioeconomico.drop('Nivel_MHI', axis=1)
y = df_socioeconomico['Nivel_MHI']

#Aplicando label encoder na variavel target
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)


In [12]:
# Criando uma função para avaliar os modelos
def avalia_modelo(modelo, X_train, y_train, X_test, y_test):
    # Treinando o modelo
    modelo.fit(X_train, y_train)

    # Fazendo as previsões
    y_pred = modelo.predict(X_test)

    # Avaliando o modelo
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

    # Retornando a acurácia do modelo
    return modelo.score(X_test, y_test)

### Utilizando um classificador "dummy" para ter uma base de comparação;

In [13]:
# Criando o modelo dummy
dummy = DummyClassifier(strategy='uniform', random_state=42)

# Aplicando a função de avaliação
dummy_u = avalia_modelo(dummy, X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

           0       0.24      0.44      0.31        48
           1       0.24      0.37      0.29        51
           2       0.64      0.37      0.47       163

    accuracy                           0.39       262
   macro avg       0.37      0.39      0.36       262
weighted avg       0.49      0.39      0.41       262

[[21 14 13]
 [11 19 21]
 [55 47 61]]


### (1) SVM - Support Vector Machine;

In [14]:
# Criando o modelo SVM
svm = SVC(kernel='rbf', C=1.0, gamma='auto', decision_function_shape='ovo', random_state=42)

# Aplicando a função de avaliação
svm_u = avalia_modelo(svm, X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        48
           1       0.00      0.00      0.00        51
           2       0.62      1.00      0.77       163

    accuracy                           0.62       262
   macro avg       0.21      0.33      0.26       262
weighted avg       0.39      0.62      0.48       262

[[  0   0  48]
 [  0   0  51]
 [  0   0 163]]
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to

### (2) Random Forest;

In [15]:
# Criando o modelo Random Forest
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# Aplicando a função de avaliação
rf_u = avalia_modelo(rf, X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

           0       0.33      0.02      0.04        48
           1       0.50      0.06      0.11        51
           2       0.62      0.97      0.76       163

    accuracy                           0.62       262
   macro avg       0.49      0.35      0.30       262
weighted avg       0.55      0.62      0.50       262

[[  1   0  47]
 [  0   3  48]
 [  2   3 158]]


### (3) MLP - Multi Layer Perceptron;

In [16]:
# Criando o modelo MLP
mlp = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000, random_state=42)

# Aplicando a função de avaliação
mlp_u = avalia_modelo(mlp, X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

           0       0.14      0.10      0.12        48
           1       0.34      0.20      0.25        51
           2       0.61      0.74      0.67       163

    accuracy                           0.52       262
   macro avg       0.37      0.35      0.35       262
weighted avg       0.47      0.52      0.49       262

[[  5   4  39]
 [  3  10  38]
 [ 27  15 121]]


### (4) XGBoost;

In [17]:
# Criando o modelo XGBoost
xgb = XGBClassifier(n_jobs=-1, random_state=42)

# Aplicando a função de avaliação
xgb_u = avalia_modelo(xgb, X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

           0       0.35      0.12      0.18        48
           1       0.36      0.18      0.24        51
           2       0.63      0.85      0.72       163

    accuracy                           0.58       262
   macro avg       0.45      0.38      0.38       262
weighted avg       0.52      0.58      0.53       262

[[  6   1  41]
 [  1   9  41]
 [ 10  15 138]]


## Treinamento dos modelos com os dados balanceados;

In [20]:
# Importando o SMOTE
from imblearn.over_sampling import SMOTE

# Criando o modelo SMOTE
smote = SMOTE(random_state=42)

# Aplicando o SMOTE nos dados de treino
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


In [21]:
# Verificando o balanceamento dos dados
print('Dados de treino:', X_train.shape, y_train.shape)
print('Dados de treino com SMOTE:', X_train_smote.shape, y_train_smote.shape)


Dados de treino: (610, 55) (610,)
Dados de treino com SMOTE: (1233, 55) (1233,)


## Aplicando GridSearch para encontrar os melhores parametros para cada modelo;

In [22]:
# Importando o GridSearch
from sklearn.model_selection import GridSearchCV

# Criando uma função para aplicar o GridSearch
def grid_search(modelo, parametros, X_train, y_train, X_test, y_test):
    # Criando o modelo
    modelo_gs = GridSearchCV(modelo, parametros, cv=5, scoring='accuracy', n_jobs=-1)

    # Treinando o modelo
    modelo_gs.fit(X_train, y_train)

    # Fazendo as previsões
    y_pred = modelo_gs.predict(X_test)

    # Avaliando o modelo
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

    # Retornando a acurácia do modelo
    return modelo_gs.score(X_test, y_test)


### Aplicando GridSearch no SVM;

In [23]:
# Definindo Modelo
svm_gs = SVC(random_state=42)

# Definindo os parametros
parametros_svm = {'C': [0.1, 1, 10, 100, 1000],
                  'gamma': ['scale', 'auto'],
                  'kernel': ['rbf', 'linear'],
                  'decision_function_shape': ['ovo', 'ovr']}

# Aplicando o GridSearch
svm_gs = grid_search(svm_gs, parametros_svm, X_train_smote, y_train_smote, X_test, y_test)


              precision    recall  f1-score   support

           0       0.19      0.15      0.16        48
           1       0.31      0.16      0.21        51
           2       0.61      0.74      0.67       163

    accuracy                           0.52       262
   macro avg       0.37      0.35      0.35       262
weighted avg       0.47      0.52      0.49       262

[[  7   1  40]
 [  5   8  38]
 [ 25  17 121]]


### Aplicando GridSearch no Random Forest;

In [24]:
# Definindo Modelo
rf_gs = RandomForestClassifier(random_state=42)

# Definindo os parametros
parametros_rf = {'n_estimators': [100, 200, 300, 400, 500],
                 'criterion': ['gini', 'entropy'],
                 'max_depth': [None, 5, 10, 15, 20],
                 'min_samples_split': [2, 5, 10],
                 'min_samples_leaf': [1, 2, 4],
                 'max_features': ['auto', 'sqrt', 'log2']}
# Aplicando o GridSearch
rf_gs = grid_search(rf_gs, parametros_rf, X_train_smote, y_train_smote, X_test, y_test)

  warn(
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this param

KeyboardInterrupt: 

### Aplicando GridSearch no MLP;

In [0]:
# Definindo Modelo
mlp_gs = MLPClassifier(random_state=42)

# Definindo os parametros
parameters_mlp = {'hidden_layer_sizes': [(10,), (50,), (100,), (10,10), (50,50), (100,100)],
              'activation': ['identity', 'logistic', 'tanh', 'relu'],
              'solver': ['lbfgs', 'sgd', 'adam'],
              'alpha': [0.01, 0.1, 1],
              'learning_rate': ['constant', 'invscaling', 'adaptive'],
              'tol': [1e-3, 1e-4, 1e-5],
              'early_stopping': [True],
              'validation_fraction': [0.1, 0.2],
              'beta_1': [0.9, 0.95],}

# Aplicando o GridSearch
mlp_gs = grid_search(mlp_gs, parameters_mlp, X_train_smote, y_train_smote, X_test, y_test)


### Aplicando GridSearch no XGBoost;

In [0]:
# Definindo Modelo
xgb_gs = XGBClassifier(random_state=42)

# Definindo os parametros
parameters_xgb = {'learning_rate': [0.01, 0.1, 0.3],
              'max_depth': [3, 5, 7],
              'n_estimators': [50, 100, 200],
              'subsample': [0.5, 0.7, 0.9],
              'colsample_bytree': [0.5, 0.7, 0.9],
              'gamma': [0, 0.1, 0.5, 1],
              'reg_alpha': [0, 0.1, 1],
              'reg_lambda': [0, 0.1, 1],
              'min_child_weight': [1, 3, 5]}

# Aplicando o GridSearch
xgb_gs = grid_search(xgb_gs, parameters_xgb, X_train_smote, y_train_smote, X_test, y_test)

### Aplicando TPOT para encontrar o melhor modelo para o problema;

In [0]:
# Importando o TPOT
from tpot import TPOTClassifier

# Criando o modelo TPOT limitando o tempo de execução em 25 minutos e modelos simples
tpot = TPOTClassifier(verbosity=2, n_jobs=-1, max_time_mins=60)

# Treinando o modelo
tpot.fit(X_train_smote, y_train_smote)

# Imprimindo os 3 melhores modelos
tpot.fitted_pipeline_

# Pegando o melhor modelo
tpot.fitted_pipeline_.steps[-1][1]

# Fazendo as previsões
y_pred_tpot = tpot.predict(X_test)

In [0]:
# Fazendo as previsões
y_pred_tpot = tpot.predict(X_test)

# Avaliando o modelo
print(classification_report(y_test, y_pred_tpot))
print(confusion_matrix(y_test, y_pred_tpot))


### Aplicando os modelos de classificação com os dados balanceados;

In [0]:
# Modelo Dummy
dummy_b = avalia_modelo(dummy, X_train_smote, y_train_smote, X_test, y_test)

In [0]:
# Modelo SVM
svm_b = (svm, X_train_smote, y_train_smote, X_test, y_test)

In [0]:
# Modelo Random Forest
rf_b = avalia_modelo(rf, X_train_smote, y_train_smote, X_test, y_test)

In [0]:
# Modelo MLP
mlp_b = avalia_modelo(mlp, X_train_smote, y_train_smote, X_test, y_test)

In [0]:
# Modelo XGBoost
xgb_b = avalia_modelo(xgb, X_train_smote, y_train_smote, X_test, y_test)

## Aplicando Tecnicas de Seleção de Features para melhorar o desempenho dos modelos;
(1) - SFS
(2) - SFFS
(3) - SFBS
(4) - SBS

In [0]:
# Importando as bibliotecas necessárias
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [0]:
# Criando o modelo SFS com o SVM e N features ( 0 < n < shape(X_train_smote) )
maxSize = np.shape(X_train_smote)[1]

sfs = SFS(svm, k_features=(1, maxSize), forward=True, floating=False, verbose=2, scoring='accuracy', cv=4, n_jobs=-1)

# Treinando o modelo
sfs.fit(X_train_smote, y_train_smote)

In [0]:
# Pegando as features selecionadas e testando o modelo
X_train_sfs = sfs.transform(X_train_smote)
X_test_sfs = sfs.transform(X_test)

# Aplicando a função de avaliação
svm_sfs = avalia_modelo(svm, X_train_sfs, y_train_smote, X_test_sfs, y_test)

In [0]:
# Criando o modelo SFFS com o SVM e N features ( 0 < n < shape(X_train_smote) )

sffs = SFS(svm, k_features=(1, maxSize), forward=True, floating=True, verbose=2, scoring='accuracy', cv=4, n_jobs=-1)

# Treinando o modelo
sffs.fit(X_train_smote, y_train_smote)


In [0]:
# Pegando as features selecionadas e testando o modelo
X_train_sffs = sffs.transform(X_train_smote)
X_test_sffs = sffs.transform(X_test)

# Aplicando a função de avaliação
svm_sffs = avalia_modelo(svm, X_train_sffs, y_train_smote, X_test_sffs, y_test)

In [0]:
# Criando o modelo SBS com o SVM
sbs = SFS(svm,k_features=3,forward=False,floating=False,scoring='accuracy',cv=4,n_jobs=-1)

# Treinando o modelo
sbs.fit(X_train_smote, y_train_smote)

In [0]:
# Pegando as features selecionadas e testando o modelo
X_train_sbs = sbs.transform(X_train_smote)
X_test_sbs = sbs.transform(X_test)

# Aplicando a função de avaliação
svm_sbs = avalia_modelo(svm, X_train_sbs, y_train_smote, X_test_sbs, y_test)


In [0]:
# Criando o modelo SFBS com o SVM
sfbs = SFS(svm,k_features=3,forward=False,floating=True,scoring='accuracy',cv=4,n_jobs=-1)

# Treinando o modelo
sfbs.fit(X_train_smote, y_train_smote)

In [0]:
# Pegando as features selecionadas e testando o modelo
X_train_sfbs = sfbs.transform(X_train_smote)
X_test_sfbs = sfbs.transform(X_test)

# Aplicando a função de avaliação
svm_sfbs = avalia_modelo(svm, X_train_sfbs, y_train_smote, X_test_sfbs, y_test)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=02993362-5ba5-4681-9d73-bfcd846e87a4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>